In [1]:
import os

In [3]:
path = 'D:\Projects\PIB Chatbot\Project_P\ScraperFiles\Files'

In [22]:
count = 0

In [23]:
for i in os.listdir(path):
    for j in os.listdir(os.path.join(path, i)):
        for k in os.listdir(os.path.join(path, i , j)):
            try:
                with open(os.path.join(path, i , j, k), 'r', encoding='utf-8') as f:
                    f.read()
                    f.close()
                    count+=1
            except Exception as e:
                print(k, 'Error' , e)
            

In [24]:
count

11020

In [16]:
from langchain.document_loaders import TextLoader
from langchain.schema import Document

directory = "D:\Projects\PIB Chatbot\Project_P\ScraperFiles\Files"
documents = []


for i in os.listdir(directory):
    for j in os.listdir(os.path.join(path, i)):
        for k in os.listdir(os.path.join(path, i , j)):
            try:
                if k.endswith(".txt"):
                    file_path = os.path.join(directory, i , j, k)
                    
                    # Create loader and load the document
                    loader = TextLoader(file_path)
                    doc = loader.load()[0]
                    
                    # Add metadata (e.g., filename or custom data)
                    doc.metadata = {"source": k, "Ministry": j, "Month":i}
                    
                    documents.append(doc)
            except Exception as e:
                print(k, 'Error' , e)

In [18]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    separators=[
        "\n\n",
        "\n"
    ]
)

In [19]:
chunks = text_splitter.split_documents(documents)

In [21]:
len(chunks)

59005

In [9]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L6-v2")

d:\Projects\PIB Chatbot\Project_P\.venv\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec # Import Pinecone and ServerlessSpec
from dotenv import load_dotenv
import os

load_dotenv()

pinecone_api_key = os.getenv("PINECONE_API_KEY")  # Add your Pinecone API key 
gemini_api_key = os.getenv("GOOGLE_API_KEY")  # Add your Google Generative AI key

pinecone_api_key = pinecone_api_key # Use your API key
pc = Pinecone(api_key=pinecone_api_key)


In [22]:
load_dotenv()

True

In [23]:
gemini_api_key = os.getenv("GOOGLE_API_KEY") 

In [7]:
index_name = "pib"
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

index = pc.Index(index_name)

In [10]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [31]:
vector_store.add_documents(chunks)

['933e13f0-bea7-46fc-b369-e3afd58ef02f',
 '76a4fc23-28aa-4db3-9c06-431a9898fe63',
 '6bd679f1-43f1-444a-ba80-8208a35f5118',
 'bc1c3351-5245-4f37-92c2-673fa7a93bf6',
 'fd9a23a8-8153-4ec1-817e-f0a5ec1472e7',
 '0e84ce6d-7d6f-406d-8c43-94518b3e9dd4',
 '65d54383-cc4d-4170-aa22-a2365d8d82d3',
 '2408bb41-9e99-498a-8452-8bc20b91ef00',
 '30eede1f-346c-44ad-9d30-876216cb84ca',
 '26004bc3-8e55-4263-94a4-fd04c8ae42df',
 '079f358c-61c9-43ab-83e8-5aa152480e23',
 'dbb5796b-094e-4617-a114-4024bb7823fe',
 'ba1ca13b-5f9e-4ba5-a81c-634f73bb5d31',
 'd85d3199-3b49-4d6b-9e47-96b94107b2c4',
 'cc3087ca-0d1c-40e0-b87e-3ff8cad4c495',
 'cea78fd5-9115-4943-b7d8-f590517a26ed',
 '5f27af15-faff-4dad-a276-fb8d104eefe5',
 '7ccf599f-c1e3-4c01-abdb-fcb1d9d7a6dd',
 '668a43ea-b7fd-4f43-91b9-a01465259d6c',
 '31f4f256-733c-4ed7-af46-b3ba87935434',
 'a96fa141-0383-4dfa-aad4-fb2c31a93089',
 'c338593e-eb7f-4365-8ec4-11624e55b6db',
 '068c4b7e-02d4-4c79-8f3a-8efdb010f9de',
 '15228002-b7ec-401b-af27-1f8d379294dc',
 '7bd65120-6bcc-

In [19]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 4, "score_threshold": 0.7},
)

In [35]:
retriever.invoke('Tell me about Vice Admiral Atul Anand?')

[Document(id='bb92199b-8683-4c11-a527-83d63a10fd16', metadata={'Ministry': 'Ministry_of_Defence', 'Month': 'Jul23html', 'source': 'Vice_Admiral_Atul_Anand_takes_over_as_Ad.txt'}, page_content='Vice Admiral Atul Anand took over as Additional Secretary, Department of Military Affairs on July 03, 2023. The post was previously held by Lt Gen Anil Puri, who superannuated on February 28, 2023.\nVice Admiral Atul Anand was commissioned into the executive branch of the Indian Navy on January 01, 1988. He is an alumnus of National Defence Academy (71st Course, Delta Squadron); Defence Services Command and Staff College, Mirpur (Bangladesh) and National Defence College, New Delhi. He has also attended the Advanced Security Cooperation course at Asia Pacific Centre of Security Studies, Hawaii, USA.'),
 Document(id='37993645-4733-4e97-b074-ca0a2e4f4592', metadata={'Ministry': 'Ministry_of_Defence', 'Month': 'Apr23html', 'source': 'VICE_ADM_ATUL_ANAND_TAKES_OVER_AS_DIRECT.txt'}, page_content='Vice 

In [27]:
from langchain_google_genai import GoogleGenerativeAI

llm = GoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=gemini_api_key,
    temperature=0.01, # HIGHER THE TEMP. LESSER THE ACCURACY OF THE MODEL
)

In [25]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)



In [36]:
ans = rag_chain.invoke("Tell me in detail about Vice Admiral Atul Anand?")

In [37]:
print(ans)

Vice Admiral Atul Anand is a highly decorated officer in the Indian Navy, having received both the Ati Vishisht Seva Medal and the Vishisht Seva Medal. He assumed the role of Director General Naval Operations on April 1st, 2023, and has held several key appointments, including command of multiple ships and important staff positions. He is an alumnus of the National Defence Academy and has a strong educational background, including a Master's in Defence Studies and an MPhil in Defence and Strategic Studies. Thanks for asking! 

